反转路网

F

In [16]:
import geopandas as gpd
import numpy as np

# 读取原始的 Shapefile 文件
original_shp = gpd.read_file('C:/Users/MI/Desktop/路网暂用/GCJ02/拼接/m3.shp')
merged_records = []  # 用于存储合并后的记录的列表

# 识别长度小于 100 的记录并提取对应的 roadid
idlist = original_shp.loc[original_shp['LENGTH'] < 150, 'OBJECTID'].tolist()

# 遍历 idlist

delList=[]
for roadid in idlist:
    # 提取记录1
    record1 = original_shp.loc[original_shp['OBJECTID'] == roadid]
    
    # 检查记录1是否存在
    if record1.empty:
        continue
    
    record1 = record1.iloc[0]
    
    fid1 = record1['FNODE']
    tid1 = record1['TNODE']
    roadname1 = record1['NAME_CHN']
    length1 = record1['LENGTH']
    fclass1 = record1['finalclass']
    geometry1 = record1['geometry']
    mesh1=record1['MESH']
    object1 = record1['OBJECTID']
    
    
    # 找到符合条件的记录2
    record2 = original_shp.loc[(original_shp['FNODE'] == tid1) & (original_shp['OBJECTID'] != roadid)]
    # 遍历记录2 DataFrame
    for index, record3 in record2.iterrows():
        fid3 = record3['FNODE']
        tid3 = record3['TNODE']
        roadname3 = record3['NAME_CHN']
        length3 = record3['LENGTH']
        fclass3 = record3['finalclass']
        geometry3 = record3['geometry']
        object3 = record3['OBJECTID']
        mesh3=record3['MESH']
        
        # 检查合并条件
        if  fclass1 == fclass3:
            # 合并记录1和记录2的属性
            merged_fid = fid1
            merged_tid = tid3
            merged_fclass = fclass1
            merged_name = roadname1
            merged_length = length1 + length3
            
            # 合并记录1和记录2的几何图形
            merged_geometry = geometry1.union(geometry3)
            
            # 创建合并后的记录
            merged_record = {
                'OBJECTID': object3,
                'FNODE': merged_fid,
                'TNODE': merged_tid,
                'finalclass': merged_fclass,
                'NAME_CHN': merged_name,
                'LENGTH': merged_length,
                'geometry': merged_geometry,
                'MESH':mesh1
            }
            
            merged_records.append(merged_record)
            delList.append(object1)
            delList.append(object3)

# 创建包含合并记录和剩余记录的新 GeoDataFrame


merged_shp = gpd.GeoDataFrame(merged_records, crs=original_shp.crs)

remaining_records = original_shp.loc[~original_shp['OBJECTID'].isin(delList)]
merged_shp = merged_shp.append(remaining_records, ignore_index=True)

# 保存合并后的 Shapefile 文件
merged_shp.to_file('C:/Users/MI/Desktop/路网暂用/GCJ02/拼接/m4.shp', driver='ESRI Shapefile', encoding='utf-8')
